# Figure 9 - CW LIFUS neuromodulatory effects on STN neuron at low acoustic intensities

Compute the temporal firing rate profiles of an STN neuron over a narrow range of low acoustic intensities, from SONIC model predictions.

### Imports

In [1]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
from PySONIC.utils import logger, Intensity2Pressure
from PySONIC.plt import CompTimeSeries, GroupedTimeSeries
from PySONIC.core import NeuronalBilayerSonophore, PulsedProtocol, AcousticDrive, Batch
from PySONIC.neurons import getPointNeuron
from PySONIC.plt import cm2inch
from utils import saveFigsAsPDF, subdirectory

logger.setLevel(logging.INFO)

### Data sub-directory

In [2]:
subdir = subdirectory('STN')

### Plot parameters

In [3]:
figindex = 9
FR_figsize = cm2inch(20, 8)
trace_figsize = cm2inch(14, 6)
fs = 12
figs = {}

### Simulation parameters

In [ ]:
a = 32e-9       # m
Fdrive = 500e3  # Hz
pp = PulsedProtocol(1.0, 1.0, 100., 1.0)
cov = 1.0
pneuron = getPointNeuron('STN')
nbls = NeuronalBilayerSonophore(a, pneuron)

## Panel A: firing rate profiles as a function of acoustic amplitude

In [ ]:
intensities = np.hstack((
        np.arange(10, 101, 10),
        np.arange(101, 131, 1),
        np.array([140])))  # W/m2
amps = Intensity2Pressure(intensities)  # Pa
queue = [([AcousticDrive(Fdrive, A), pp, cov, 'sonic', None], {'outputdir': subdir}) for A in amps]
batch = Batch(nbls.getOutput, queue)
outputs = batch.run(mpi=True, loglevel=logger.level)
fig = CompTimeSeries(outputs, 'FR').render(
    patches='none', cmap='Oranges', trange=(0, 1.0), figsize=FR_figsize, fs=fs)
figs['a'] = fig

## Panel B: charge density profiles for characteristic acoustic amplitudes

In [ ]:
intensities = np.array([100, 114, 123])  # W/m2
amps = Intensity2Pressure(intensities)  # Pa
outputs = [nbls.getOutput(AcousticDrive(Fdrive, A), pp, cov, 'sonic', None, outputdir=subdir) for A in amps]
figtraces = GroupedTimeSeries(outputs, pltscheme={'Q_m': ['Qm']}).render(fs=fs)
for (I, A), fig in zip(zip(intensities, amps), figtraces):
    fig.axes[0].set_title(f'{A * 1e-3:.2f} kPa ({I:.0f} W/m2)')
    figs[f'b_Qtrace_{I:.0f}_W_per_m2'] = fig

### Save figure panels

Save figure panels as **pdf** in the *figs* sub-folder:

In [ ]:
saveFigsAsPDF(figs, figindex)